---
# ASV raw data processing

title: P1706_ASV   
author: Ralph  
date: 2023-01-20 
input: csv files  
output: csv files  

The ASV 16S and 18SV9 files are used here. The sample names are renamed similar to that of the mzxml file names. Also, the taxonomic information for each feature is combined with the OTU table

---

## Step 1: Load packages and set working directory and shared files

In [ ]:
install.packages(c('dplyr','tidyr'))

In [2]:
library(dplyr)
library(tidyr)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




In [ ]:
#copy and paste the filepath here:

In [4]:
setwd(normalizePath(readline("Enter the path of the folder with input files: "),"/",mustWork=FALSE))

Enter the path of the folder with input files: G:\My Drive\CCE DATA\P1706_ASV


In [12]:
# Jointname files for renaming ASV samples (Ralph manually curated to match ASV names with feature table and metadata)
nameindex <- na.omit(read.csv('sample_ID_index.csv'))
nameindex <- nameindex %>% na_if("") %>% na.omit
head(nameindex)

,name1,name2,name3,name4
,<chr>,<chr>,<chr>,<chr>
3,P1706_SBB__00_1_C,P1706_073,CCE_P1706_3,CCE_P1706_3.mzxml
4,P1706_SBB__00_1_C2,P1706_073,CCE_P1706_4,CCE_P1706_4.mzxml
5,P1706_SBB__00_2_C,P1706_074,CCE_P1706_5,CCE_P1706_5.mzxml
6,P1706_SBB__00_2_C2,P1706_074,CCE_P1706_6,CCE_P1706_6.mzxml
9,P1706_T1_01_100_6_A,P1706_001,CCE_P1706_9,CCE_P1706_9.mzxml
10,P1706_T1_01_050_5_A,P1706_002,CCE_P1706_10,CCE_P1706_10.mzxml


These 4 columns represent the different names for each file. For ex: 4th column: 'name4' is the file name obtained from mass spectrometry analysis 

In [11]:
metadata <- read.csv('metadata_CCE_updatedRRT.csv')
head(metadata)

,filename,ATTRIBUTE_x,ATTRIBUTE_y,ATTRIBUTE_z,ATTRIBUTE_Spot_size,ATTRIBUTE_Filament_Possition,ATTRIBUTE_Sample_Number,ATTRIBUTE_Date,ATTRIBUTE_Cast,ATTRIBUTE_Event_Number,⋯,ATTRIBUTE_NH4,ATTRIBUTE_PO4_binned,ATTRIBUTE_NO2_binned,ATTRIBUTE_NO3_binned,ATTRIBUTE_NH4_binned,ATTRIBUTE_DOC,ATTRIBUTE_Si_excess,ATTRIBUTE_C_N_Molar,ATTRIBUTE_BCP_ugC_L_d,ATTRIBUTE_Primary_Production_ug_L_day
,<chr>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<int>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,Blank_CCE_P1706_1.mzxml,0.0,0.0,0.0,1,Blank,1,20170602,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
2,Blank_CCE_P1706_2.mzxml,0.0,0.0,0.0,1,Blank,2,20170602,NA,NA,⋯,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA
3,CCE_P1706_3.mzxml,-80.1,85.1,101.5,1,Control,3,20170602,2,NA,⋯,0.498,3.6,1.9,13,0.5,NA,NA,NA,NA,NA
4,CCE_P1706_4.mzxml,-80.1,85.1,101.5,1,Control,4,20170602,2,NA,⋯,0.498,3.6,1.9,13,0.5,NA,NA,NA,NA,NA
5,CCE_P1706_5.mzxml,-80.1,85.1,102.5,1,Control,5,20170602,2,NA,⋯,0.022,3.4,0.3,20,0.0,NA,NA,NA,NA,NA
6,CCE_P1706_6.mzxml,-80.1,85.1,102.5,1,Control,6,20170602,2,NA,⋯,0.022,3.4,0.3,20,0.0,NA,NA,NA,NA,NA


## Step 2: 16S processing

Load Files:

In [49]:
table16S <- read.csv('16S_ASV_table.csv')
ASVtax16S <- read.csv('16S_ASV_taxonomy.csv')

In [50]:
head(table16S)
dim(table16S)

,X,P1706_T1_01_100_6_A,P1706_T1_01_050_5_A,P1706_T1_01_030_4_A,P1706_T1_01_020_3_A,P1706_T1_01_010_2_A,P1706_T1_01_000_1_A,P1706_T1_03_100_6_B,P1706_T1_03_050_5_B,P1706_T1_03_030_4_B,⋯,P1706_B6_6_040_2_C,P1706_B7_7_047_3_C,P1706_B7_7_035_2_C,P1706_B7_7_010_1_C,P1706_B8_8_051_3_C,P1706_B8_8_040_2_C,P1706_B8_8_015_1_C,P1706_B9_9_051_3_C,P1706_B9_9_040_2_C,P1706_B9_9_015_1_C
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,A_682a528ad8f32005b9636c913646746b,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,A_82702b8c1700a6d17e44335686b3979f,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
3,A_e3abbabd0a089c4f55c21439dc79dca2,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
4,A_c4df1e667d9c8cc8f0b63ed759c624fa,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
5,A_7fe3ef3730dce25ba70e66e8e68a46ae,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
6,A_2d140f0d6e674015f414108052ee7b96,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


[1] 7534  179

In [51]:
head(ASVtax16S)
dim(ASVtax16S)

,Feature.ID,Taxon,Confidence
,<chr>,<chr>,<dbl>
1,b184b60cdae98aed6f7b56fc0e8b0d3f,D_0__Bacteria;D_1__Bacteroidetes;D_2__Bacteroidia;D_3__Chitinophagales;D_4__Saprospiraceae;D_5__Aureispira;D_6__uncultured bacterium,0.8270177
2,cfb1cd351c8c415dc2b70ecb58678d1d,D_0__Bacteria;D_1__Planctomycetes;D_2__Planctomycetacia;D_3__Pirellulales;D_4__Pirellulaceae;D_5__Pir4 lineage,0.8612480
3,e18dc1931cfa8bf1005b031dd31dcb28,D_0__Bacteria;D_1__Lentisphaerae;D_2__Lentisphaeria;D_3__Victivallales;D_4__BD2-3;D_5__uncultured Lentisphaerae bacterium;D_6__uncultured Lentisphaerae bacterium,0.9832359
4,a505046e7e3debcf71b5bda0d802d1fa,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Francisellales;D_4__Francisellaceae,0.8633116
5,02299cecf4b39b8a89d99539e7f8e399,D_0__Bacteria;D_1__Proteobacteria;D_2__Deltaproteobacteria;D_3__Bdellovibrionales;D_4__Bdellovibrionaceae;D_5__OM27 clade;D_6__uncultured bacterium,0.7528309
6,98afd1d765938c86754b7d8991612720,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria;D_3__Alteromonadales;D_4__Idiomarinaceae;D_5__Idiomarina,0.9999959


[1] 7534    3

Cleaning up the ASV table to match taxonomy file:

In [52]:
##  Rename column X to Feature.ID to match other tax file
table16S <- table16S %>% dplyr::rename('Feature.ID' = 'X')
# Remove the A_ header in all of the names in the 16S table to match tax file
table16S$Feature.ID <- gsub('A_', '', as.character(table16S$Feature.ID))

head(table16S)

,Feature.ID,P1706_T1_01_100_6_A,P1706_T1_01_050_5_A,P1706_T1_01_030_4_A,P1706_T1_01_020_3_A,P1706_T1_01_010_2_A,P1706_T1_01_000_1_A,P1706_T1_03_100_6_B,P1706_T1_03_050_5_B,P1706_T1_03_030_4_B,⋯,P1706_B6_6_040_2_C,P1706_B7_7_047_3_C,P1706_B7_7_035_2_C,P1706_B7_7_010_1_C,P1706_B8_8_051_3_C,P1706_B8_8_040_2_C,P1706_B8_8_015_1_C,P1706_B9_9_051_3_C,P1706_B9_9_040_2_C,P1706_B9_9_015_1_C
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,682a528ad8f32005b9636c913646746b,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,82702b8c1700a6d17e44335686b3979f,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
3,e3abbabd0a089c4f55c21439dc79dca2,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
4,c4df1e667d9c8cc8f0b63ed759c624fa,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
5,7fe3ef3730dce25ba70e66e8e68a46ae,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
6,2d140f0d6e674015f414108052ee7b96,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


Next Step is to match names in the ASV table to the MS data file names associated with that sample. For Cycles, BBL and SBB, duplicates were taken. <br>
We therefore have to create duplicates of our ASV data to associate with the MS data. 

In [53]:
table16S <- table16S %>% mutate(
    across(
    .cols = contains(c('C1', 'C2', 'C3', 'C4', 'SBB', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9'),
                     ignore.case = FALSE),
    .names = '{.col}2'
  )
)

In [54]:
head(table16S)

,Feature.ID,P1706_T1_01_100_6_A,P1706_T1_01_050_5_A,P1706_T1_01_030_4_A,P1706_T1_01_020_3_A,P1706_T1_01_010_2_A,P1706_T1_01_000_1_A,P1706_T1_03_100_6_B,P1706_T1_03_050_5_B,P1706_T1_03_030_4_B,⋯,P1706_B6_6_040_2_C2,P1706_B7_7_047_3_C2,P1706_B7_7_035_2_C2,P1706_B7_7_010_1_C2,P1706_B8_8_051_3_C2,P1706_B8_8_040_2_C2,P1706_B8_8_015_1_C2,P1706_B9_9_051_3_C2,P1706_B9_9_040_2_C2,P1706_B9_9_015_1_C2
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,682a528ad8f32005b9636c913646746b,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,82702b8c1700a6d17e44335686b3979f,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
3,e3abbabd0a089c4f55c21439dc79dca2,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
4,c4df1e667d9c8cc8f0b63ed759c624fa,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
5,7fe3ef3730dce25ba70e66e8e68a46ae,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
6,2d140f0d6e674015f414108052ee7b96,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


In [55]:
#colnames(table16S) to look at the duplicates

Now, we merge these names with the manually curated jointname files to reflect the associated metadata datafile.

In [56]:
match(nameindex[, "name1"], names(table16S))

[1]  74 251  75 252   2   3   4   5   6   7   8   9  10  11  12  13  14  15
 [19]  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32 180
 [37]  33 181  34 182  35 183  36 184  37 185  38 186  39 187  40 188  41 189
 [55]  42 190  43 191  44 192  45 193  46 194  47 195  48 196  49 197  50 198
 [73]  51 199  52 200  53 201  54 202  55 203  56 204  57 205  58 206  59 207
 [91]  60 208  61 209  62 210  63 211  64 212  65 213  66 214  67 215  68 216
[109]  69 217  70 218  71 219  72 220  73 221  76  77  78  79  80  81  82  83
[127]  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
[145] 102 103 104 105 106 222 107 223 108 224 109 225 110 226 111 227 112 228
[163] 113 229 114 230 115 231 116 232 117 233 118 234 119 235 120 236 121 237
[181] 122 238 123 239 124 125 126 127 128 129 130 131 132 133 134 135 136 137
[199] 138 139 140 141 142 240 143 241 144 242 145 243 146 244 147 245 148 265
[217] 149 266 150 267 151 268 152 246 153 247 154 248 155 249 156 250 157 253
[235] 158 254 159 255 160 256 161 257 162 258 163 259 164 260 165 261 166 262
[253] 167 263 168 264 169 269 170 270 171 271 172 272 173 273 174 274 175 275
[271] 176 276 177 277 178 278 179 279

In [57]:
names(table16S)[match(nameindex[, "name1"], names(table16S))] = nameindex[, "name4"]
head(table16S, n=2)

,Feature.ID,CCE_P1706_9.mzxml,CCE_P1706_10.mzxml,CCE_P1706_11.mzxml,CCE_P1706_12.mzxml,CCE_P1706_13.mzxml,CCE_P1706_14.mzxml,CCE_P1706_15.mzxml,CCE_P1706_16.mzxml,CCE_P1706_17.mzxml,⋯,CCE_P1706_264.mzxml,CCE_P1706_268.mzxml,CCE_P1706_270.mzxml,CCE_P1706_272.mzxml,CCE_P1706_274.mzxml,CCE_P1706_276.mzxml,CCE_P1706_278.mzxml,CCE_P1706_280.mzxml,CCE_P1706_282.mzxml,CCE_P1706_284.mzxml
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,682a528ad8f32005b9636c913646746b,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0
2,82702b8c1700a6d17e44335686b3979f,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,0,0


Now the ASV table matches the names within the metadata file. Next, we must select only files associated with Cycles.
We also need to transpose the metadata file in order to merge it with the ASV table.

In [58]:
#Select only Cycles in the metadata file
md <- subset(metadata, ATTRIBUTE_Sample_Type == "Cycle")
head(md, n=3)

,filename,ATTRIBUTE_x,ATTRIBUTE_y,ATTRIBUTE_z,ATTRIBUTE_Spot_size,ATTRIBUTE_Filament_Possition,ATTRIBUTE_Sample_Number,ATTRIBUTE_Date,ATTRIBUTE_Cast,ATTRIBUTE_Event_Number,⋯,ATTRIBUTE_NH4,ATTRIBUTE_PO4_binned,ATTRIBUTE_NO2_binned,ATTRIBUTE_NO3_binned,ATTRIBUTE_NH4_binned,ATTRIBUTE_DOC,ATTRIBUTE_Si_excess,ATTRIBUTE_C_N_Molar,ATTRIBUTE_BCP_ugC_L_d,ATTRIBUTE_Primary_Production_ug_L_day
,<chr>,<dbl>,<dbl>,<dbl>,<int>,<chr>,<int>,<int>,<int>,<int>,⋯,<dbl>,<dbl>,<dbl>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
39,CCE_P1706_39.mzxml,-101.419,101.956,109.445,1,Cycle_1,39,20170609,16,216,⋯,0.064,1.9,0.2,25,0.1,48.4,0.0000000,6.31,0.2190658,NA
40,CCE_P1706_40.mzxml,-101.419,101.956,109.445,1,Cycle_1,40,20170609,16,216,⋯,0.064,1.9,0.2,25,0.1,48.4,0.0000000,6.31,0.2190658,NA
41,CCE_P1706_41.mzxml,-101.419,101.956,110.445,1,Cycle_1,41,20170609,16,216,⋯,0.456,1.9,0.5,24,0.5,52.2,-0.7057274,6.75,0.2680229,0.001025041


In [59]:
#Transpose metadata file 
rownames(md) <- md$filename
md <- select(md, -c(filename))
md <- (t(md))
head(md,n=3)

,CCE_P1706_39.mzxml,CCE_P1706_40.mzxml,CCE_P1706_41.mzxml,CCE_P1706_42.mzxml,CCE_P1706_43.mzxml,CCE_P1706_44.mzxml,CCE_P1706_45.mzxml,CCE_P1706_46.mzxml,CCE_P1706_47.mzxml,CCE_P1706_48.mzxml,⋯,CCE_P1706_221.mzxml,CCE_P1706_222.mzxml,CCE_P1706_223.mzxml,CCE_P1706_224.mzxml,CCE_P1706_225.mzxml,CCE_P1706_226.mzxml,CCE_P1706_227.mzxml,CCE_P1706_228.mzxml,CCE_P1706_229.mzxml,CCE_P1706_230.mzxml
ATTRIBUTE_x,-101.419,-101.419,-101.419,-101.419,-101.419,-101.419,-101.419,-101.419,-101.419,-101.419,⋯,-139.636,-139.636,-139.636,-139.636,-139.636,-139.636,-139.636,-139.636,-139.636,-139.636
ATTRIBUTE_y,101.9560,101.9560,101.9560,101.9560,101.9560,101.9560,101.9560,101.9560,101.9560,101.9560,⋯,88.6513,88.6513,88.6513,88.6513,88.6513,88.6513,88.6513,88.6513,88.6513,88.6513
ATTRIBUTE_z,109.445,109.445,110.445,110.445,111.444,111.444,111.884,111.884,112.344,112.344,⋯,110.445,110.445,111.444,111.444,111.884,111.884,112.344,112.344,112.786,112.786


In [60]:
#Fix rownames to column, create dataframe
Feature.ID <- rownames(md)
rownames(md) <- NULL
md <- as.data.frame(cbind(Feature.ID, md))
head(md, n=3)
dim(md)

,Feature.ID,CCE_P1706_39.mzxml,CCE_P1706_40.mzxml,CCE_P1706_41.mzxml,CCE_P1706_42.mzxml,CCE_P1706_43.mzxml,CCE_P1706_44.mzxml,CCE_P1706_45.mzxml,CCE_P1706_46.mzxml,CCE_P1706_47.mzxml,⋯,CCE_P1706_221.mzxml,CCE_P1706_222.mzxml,CCE_P1706_223.mzxml,CCE_P1706_224.mzxml,CCE_P1706_225.mzxml,CCE_P1706_226.mzxml,CCE_P1706_227.mzxml,CCE_P1706_228.mzxml,CCE_P1706_229.mzxml,CCE_P1706_230.mzxml
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,ATTRIBUTE_x,-101.419,-101.419,-101.419,-101.419,-101.419,-101.419,-101.419,-101.419,-101.419,⋯,-139.636,-139.636,-139.636,-139.636,-139.636,-139.636,-139.636,-139.636,-139.636,-139.636
2,ATTRIBUTE_y,101.9560,101.9560,101.9560,101.9560,101.9560,101.9560,101.9560,101.9560,101.9560,⋯,88.6513,88.6513,88.6513,88.6513,88.6513,88.6513,88.6513,88.6513,88.6513,88.6513
3,ATTRIBUTE_z,109.445,109.445,110.445,110.445,111.444,111.444,111.884,111.884,112.344,⋯,110.445,110.445,111.444,111.444,111.884,111.884,112.344,112.344,112.786,112.786


[1]  37 145

In [61]:
# Make table16S characters to match metadata file to allow them to merge
table16S <- table16S %>% mutate_if(is.numeric, as.character)

In [62]:
#Remove files that don't have both metadata and ASV files associated with itself
common_column_names <- intersect(names(table16S), names(md))
table16S <- table16S[, common_column_names]
md <- md[, common_column_names]
head(md,3)
dim(md)

,Feature.ID,CCE_P1706_39.mzxml,CCE_P1706_41.mzxml,CCE_P1706_43.mzxml,CCE_P1706_45.mzxml,CCE_P1706_47.mzxml,CCE_P1706_49.mzxml,CCE_P1706_51.mzxml,CCE_P1706_53.mzxml,CCE_P1706_55.mzxml,⋯,CCE_P1706_210.mzxml,CCE_P1706_212.mzxml,CCE_P1706_214.mzxml,CCE_P1706_216.mzxml,CCE_P1706_218.mzxml,CCE_P1706_220.mzxml,CCE_P1706_222.mzxml,CCE_P1706_224.mzxml,CCE_P1706_228.mzxml,CCE_P1706_230.mzxml
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,ATTRIBUTE_x,-101.419,-101.419,-101.419,-101.419,-101.419,-101.419,-103.515,-103.515,-103.515,⋯,-141.430,-141.430,-141.430,-141.430,-141.430,-139.636,-139.636,-139.636,-139.636,-139.636
2,ATTRIBUTE_y,101.9560,101.9560,101.9560,101.9560,101.9560,101.9560,100.0030,100.0030,100.0030,⋯,86.9585,86.9585,86.9585,86.9585,86.9585,88.6513,88.6513,88.6513,88.6513,88.6513
3,ATTRIBUTE_z,109.445,110.445,111.444,111.884,112.344,112.786,109.445,110.445,111.444,⋯,110.445,111.444,111.884,112.344,112.786,109.445,110.445,111.444,112.344,112.786


[1]  37 143

Now we can merge the taxonomic information and ASV table by 'Feature.ID'. We also can merge the metadata file now.

In [63]:
#Merge the taxonomy table 
ASV16S <- merge(table16S, ASVtax16S, by = 'Feature.ID', all.x = TRUE)
#Merge metadata file and ASV file
ASV16S <- bind_rows(ASV16S, md)

head(ASV16S)
dim(ASV16S)

,Feature.ID,CCE_P1706_39.mzxml,CCE_P1706_41.mzxml,CCE_P1706_43.mzxml,CCE_P1706_45.mzxml,CCE_P1706_47.mzxml,CCE_P1706_49.mzxml,CCE_P1706_51.mzxml,CCE_P1706_53.mzxml,CCE_P1706_55.mzxml,⋯,CCE_P1706_214.mzxml,CCE_P1706_216.mzxml,CCE_P1706_218.mzxml,CCE_P1706_220.mzxml,CCE_P1706_222.mzxml,CCE_P1706_224.mzxml,CCE_P1706_228.mzxml,CCE_P1706_230.mzxml,Taxon,Confidence
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,00208b62ef66aa36e069777b853c8f17,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,D_0__Bacteria;D_1__Proteobacteria;D_2__Deltaproteobacteria;D_3__SAR324 clade(Marine group B),0.8284995
2,00232f374cfc930f3fffbaef0d0c6d80,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,D_0__Bacteria;D_1__Proteobacteria;D_2__Gammaproteobacteria,1.0000000
3,0023a8c071538909ed3386e4e1d52278,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,D_0__Bacteria;D_1__Bacteroidetes;D_2__Ignavibacteria;D_3__OPB56;D_4__uncultured organism;D_5__uncultured organism;D_6__uncultured organism,0.7360358
4,00256e2eb0d211a7498c3e8b3170cedd,0,7,0,0,0,0,2,0,0,⋯,0,0,0,1,0,0,0,0,D_0__Bacteria;D_1__Marinimicrobia (SAR406 clade),0.9999943
5,002ae12276a5da377dd6f7d5452a388a,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,D_0__Bacteria;D_1__Bacteroidetes;D_2__Bacteroidia;D_3__Flavobacteriales;D_4__NS9 marine group,0.9964438
6,002c14075180e4484913e02e02dead50,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,D_0__Bacteria;D_1__Bacteroidetes;D_2__Bacteroidia;D_3__Flavobacteriales;D_4__Crocinitomicaceae;D_5__Crocinitomix;D_6__uncultured Bacteroidetes/Chlorobi group bacterium,0.9999990


[1] 7571  145

Now we have the clean up the Taxon column

In [64]:
#Remove the "D_0__" header in the Taxon column
ASV16S$Taxon <- gsub('D_0__', '', as.character(ASV16S$Taxon))
ASV16S$Taxon <- gsub('D_1_', '', as.character(ASV16S$Taxon))
ASV16S$Taxon <- gsub('D_2_', '', as.character(ASV16S$Taxon))
ASV16S$Taxon <- gsub('D_3_', '', as.character(ASV16S$Taxon))
ASV16S$Taxon <- gsub('D_4_', '', as.character(ASV16S$Taxon))
ASV16S$Taxon <- gsub('D_5_', '', as.character(ASV16S$Taxon))
ASV16S$Taxon <- gsub('D_6_', '', as.character(ASV16S$Taxon))
ASV16S$Taxon <- gsub(';', '', as.character(ASV16S$Taxon))

In [65]:
#create a shared name column for later Cytoscape processing
ASV16S$'shared name' <- ASV16S$Taxon

head(ASV16S)

,Feature.ID,CCE_P1706_39.mzxml,CCE_P1706_41.mzxml,CCE_P1706_43.mzxml,CCE_P1706_45.mzxml,CCE_P1706_47.mzxml,CCE_P1706_49.mzxml,CCE_P1706_51.mzxml,CCE_P1706_53.mzxml,CCE_P1706_55.mzxml,⋯,CCE_P1706_216.mzxml,CCE_P1706_218.mzxml,CCE_P1706_220.mzxml,CCE_P1706_222.mzxml,CCE_P1706_224.mzxml,CCE_P1706_228.mzxml,CCE_P1706_230.mzxml,Taxon,Confidence,shared name
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
1,00208b62ef66aa36e069777b853c8f17,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,Bacteria_Proteobacteria_Deltaproteobacteria_SAR324 clade(Marine group B),0.8284995,Bacteria_Proteobacteria_Deltaproteobacteria_SAR324 clade(Marine group B)
2,00232f374cfc930f3fffbaef0d0c6d80,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,Bacteria_Proteobacteria_Gammaproteobacteria,1.0000000,Bacteria_Proteobacteria_Gammaproteobacteria
3,0023a8c071538909ed3386e4e1d52278,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,Bacteria_Bacteroidetes_Ignavibacteria_OPB56_uncultured organism_uncultured organism_uncultured organism,0.7360358,Bacteria_Bacteroidetes_Ignavibacteria_OPB56_uncultured organism_uncultured organism_uncultured organism
4,00256e2eb0d211a7498c3e8b3170cedd,0,7,0,0,0,0,2,0,0,⋯,0,0,1,0,0,0,0,Bacteria_Marinimicrobia (SAR406 clade),0.9999943,Bacteria_Marinimicrobia (SAR406 clade)
5,002ae12276a5da377dd6f7d5452a388a,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,Bacteria_Bacteroidetes_Bacteroidia_Flavobacteriales_NS9 marine group,0.9964438,Bacteria_Bacteroidetes_Bacteroidia_Flavobacteriales_NS9 marine group
6,002c14075180e4484913e02e02dead50,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,Bacteria_Bacteroidetes_Bacteroidia_Flavobacteriales_Crocinitomicaceae_Crocinitomix_uncultured Bacteroidetes/Chlorobi group bacterium,0.9999990,Bacteria_Bacteroidetes_Bacteroidia_Flavobacteriales_Crocinitomicaceae_Crocinitomix_uncultured Bacteroidetes/Chlorobi group bacterium


Deconcatenate the Taxon Column: <br>
0 is domain, 1 is phylum, 2 is class, 3 is order, 4 is family, 5 is genus, 6 is species

In [67]:
ASV16S <- separate(data = ASV16S, col = Taxon, into = c('Domain16S', 'Phylum16S', 
                                                        'Class16S', 'Order16S', 
                                                        'Family16S', 'Genus16S',
                                                        'Species16S'), sep = "_")
head(ASV16S)
dim(ASV16S)

Warning message:
"Expected 7 pieces. Additional pieces discarded in 181 rows [27, 56, 92, 185, 193, 214, 221, 232, 242, 246, 274, 289, 411, 470, 472, 487, 513, 530, 614, 659, ...]."
Warning message:
"Expected 7 pieces. Missing pieces filled with `NA` in 4437 rows [1, 2, 4, 5, 9, 10, 11, 12, 13, 17, 18, 21, 23, 25, 28, 29, 30, 31, 32, 33, ...]."


,Feature.ID,CCE_P1706_39.mzxml,CCE_P1706_41.mzxml,CCE_P1706_43.mzxml,CCE_P1706_45.mzxml,CCE_P1706_47.mzxml,CCE_P1706_49.mzxml,CCE_P1706_51.mzxml,CCE_P1706_53.mzxml,CCE_P1706_55.mzxml,⋯,CCE_P1706_230.mzxml,Domain16S,Phylum16S,Class16S,Order16S,Family16S,Genus16S,Species16S,Confidence,shared name
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<chr>
1,00208b62ef66aa36e069777b853c8f17,0,0,0,0,0,0,0,0,0,⋯,0,Bacteria,Proteobacteria,Deltaproteobacteria,SAR324 clade(Marine group B),NA,NA,NA,0.8284995,Bacteria_Proteobacteria_Deltaproteobacteria_SAR324 clade(Marine group B)
2,00232f374cfc930f3fffbaef0d0c6d80,0,0,0,0,0,0,0,0,0,⋯,0,Bacteria,Proteobacteria,Gammaproteobacteria,NA,NA,NA,NA,1.0000000,Bacteria_Proteobacteria_Gammaproteobacteria
3,0023a8c071538909ed3386e4e1d52278,0,0,0,0,0,0,0,0,0,⋯,0,Bacteria,Bacteroidetes,Ignavibacteria,OPB56,uncultured organism,uncultured organism,uncultured organism,0.7360358,Bacteria_Bacteroidetes_Ignavibacteria_OPB56_uncultured organism_uncultured organism_uncultured organism
4,00256e2eb0d211a7498c3e8b3170cedd,0,7,0,0,0,0,2,0,0,⋯,0,Bacteria,Marinimicrobia (SAR406 clade),NA,NA,NA,NA,NA,0.9999943,Bacteria_Marinimicrobia (SAR406 clade)
5,002ae12276a5da377dd6f7d5452a388a,0,0,0,0,0,0,0,0,0,⋯,0,Bacteria,Bacteroidetes,Bacteroidia,Flavobacteriales,NS9 marine group,NA,NA,0.9964438,Bacteria_Bacteroidetes_Bacteroidia_Flavobacteriales_NS9 marine group
6,002c14075180e4484913e02e02dead50,0,0,0,0,0,0,0,0,0,⋯,0,Bacteria,Bacteroidetes,Bacteroidia,Flavobacteriales,Crocinitomicaceae,Crocinitomix,uncultured Bacteroidetes/Chlorobi group bacterium,0.9999990,Bacteria_Bacteroidetes_Bacteroidia_Flavobacteriales_Crocinitomicaceae_Crocinitomix_uncultured Bacteroidetes/Chlorobi group bacterium


[1] 7571  152

Finished 16S ASV processing. Now, we can write it into .csv file

In [48]:
write.csv(ASV16S, 'ASV16S_With_Metadata_dups.csv')

## Step 3: 18SV9 processing

Here, we repeat the same process as for ASV 16S with some little changes.

Load Files (make sure to run Step 1):

In [74]:
table18SV9 <- read.csv('18SV9_ASV_table.csv')
ASVtax18SV9 <- read.csv('18SV9_ASV_taxonomy.csv')

In [75]:
head(table18SV9)
dim(table18SV9)

,X,P1706_001,P1706_002,P1706_003,P1706_004,P1706_005,P1706_006,P1706_007,P1706_008,P1706_009,⋯,P1706_170,P1706_171,P1706_172,P1706_173,P1706_174,P1706_175,P1706_176,P1706_177,P1706_178,P1706_179
,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,⋯,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,16e3d4f1cb72c753aa6420206110abec,16,116,106,588,2016,1777,0,32,727,⋯,436,1523,2270,7665,1540,937,575,2293,2895,868
2,6fdbb8c63fe109f1beeb14f76c30e678,18,70,208,401,1473,1644,33,268,399,⋯,0,0,0,90,0,0,49,0,160,118
3,d1b754a61547771e95bc457845b76031,30,0,0,0,13,0,0,0,0,⋯,118,80,94,156,124,72,446,49,98,555
4,ddabd94ae4f697d543d0d86c72539d44,596,1202,2573,387,17,87,0,4308,6415,⋯,4574,249,198,4333,2583,12900,55,1151,3478,75
5,1a2206bc89d2ea0b2eebd0b6cf6f0659,0,0,0,0,381,968,0,0,0,⋯,15684,790,2715,799,0,6,136,18,210,54
6,1e2166cbb9194f76fecce38bfad61555,0,0,123,69,141,549,7,0,0,⋯,2284,4901,1824,3346,0,0,131,145,4,3125


[1] 10282   179

In [76]:
head(ASVtax18SV9)
dim(ASVtax18SV9)

,X,Feature.ID,Taxon,Confidence
,<int>,<chr>,<chr>,<dbl>
1,1,16e3d4f1cb72c753aa6420206110abec,Eukaryota;Stramenopiles;Ochrophyta;Bacillariophyta;Bacillariophyta_X;Polar-centric-Mediophyceae;Chaetoceros;Chaetoceros_rostratus;,0.9954440
2,2,6fdbb8c63fe109f1beeb14f76c30e678,Eukaryota;Stramenopiles;Ochrophyta;Bacillariophyta;Bacillariophyta_X;Polar-centric-Mediophyceae;Chaetoceros;Chaetoceros_rostratus;,0.9871728
3,3,d1b754a61547771e95bc457845b76031,Eukaryota;Stramenopiles;Ochrophyta;Bacillariophyta;Bacillariophyta_X;Raphid-pennate;Pseudo-nitzschia;Pseudo-nitzschia_australis;,0.9895452
4,4,ddabd94ae4f697d543d0d86c72539d44,Eukaryota;Opisthokonta;Metazoa;Arthropoda;Crustacea;Maxillopoda;Pseudocalanus;Pseudocalanus_elongatus;,0.8978281
5,5,1a2206bc89d2ea0b2eebd0b6cf6f0659,Eukaryota;Opisthokonta;Metazoa;Urochordata;Urochordata_X;Appendicularia;Oikopleura;Oikopleura_sp.;,0.9400966
6,6,1e2166cbb9194f76fecce38bfad61555,Eukaryota;Opisthokonta;Metazoa;Arthropoda;Crustacea;Maxillopoda;Paracalanus;Paracalanus_parvus;,0.9977327


[1] 10282     4

Cleaning up the ASV file to match the taxonomy file:

In [77]:
#Rename column X to Feature.ID to match other excel file
table18SV9 <- rename(table18SV9, 'Feature.ID' = 'X')
#Remove X Column
ASVtax18SV9 <- subset(ASVtax18SV9, select = -c(X))

Next Step is to match names in the ASV table to the MS data file names associated with that sample. 
For the 18SV9 data, we have to first convert the names to the names of the 16S data

In [78]:
#Remove duplicated names in jointnames2 to allow merge
index18S <- nameindex[!duplicated(nameindex$name2),]
#Matching 18S ASV names to 16S ASV names
names(table18SV9)[match(index18S[, "name2"], names(table18SV9))] = index18S[, "name1"]

For Cycles, BBL and SBB, duplicates were taken. 
We therefore have to create duplicates of our ASV data to associate with the MS data.

In [79]:
#Repeat columns that have duplicate PPL samples (this way both duplicates will have ASV information tied to them)
table18SV9 <- table18SV9 %>% mutate(
  across(
    .cols = contains(c('C1', 'C2', 'C3', 'C4', 'SBB', 'B1', 'B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B9'),
                     ignore.case = FALSE),
    .names = '{.col}2'
  )
)

Now, we merge these names with the manually curated jointname files to reflect the associated metadata datafile.

In [80]:
# Rename ASV18SV9 names with feature table / metadata names
match(nameindex[, "name1"], names(table18SV9))
names(table18SV9)[match(nameindex[, "name1"], names(table18SV9))] = nameindex[, "name4"]

[1]  74 251  75 252   2   3   4   5   6   7   8   9  10  11  12  13  14  15
 [19]  16  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32 180
 [37]  33 181  34 182  35 183  36 184  37 185  38 186  39 187  40 188  41 189
 [55]  42 190  43 191  44 192  45 193  46 194  47 195  48 196  49 197  50 198
 [73]  51 199  52 200  53 201  54 202  55 203  56 204  57 205  58 206  59 207
 [91]  60 208  61 209  62 210  63 211  64 212  65 213  66 214  67 215  68 216
[109]  69 217  70 218  71 219  72 220  73 221  76  77  78  79  80  81  82  83
[127]  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98  99 100 101
[145] 102 103 104 105 106 222 107 223 108 224 109 225 110 226 111 227 112 228
[163] 113 229 114 230 115 231 116 232 117 233 118 234 119 235 120 236 121 237
[181] 122 238 123 239 124 125 126 127 128 129 130 131 132 133 134 135 136 137
[199] 138 139 140 141 142 240 143 241 144 242 145 243 146 244 147 245 148 265
[217] 149 266 150 267 151 268 152 246 153 247 154 248 155 249 156 250 157 253
[235] 158 254 159 255 160 256 161 257 162 258 163 259 164 260 165 261 166 262
[253] 167 263 168 264 169 269 170 270 171 271 172 272 173 273 174 274 175 275
[271] 176 276 177 277 178 278 179 279

Now the ASV table matches the names within the metadata file. 
Next, we must select only files associated with Cycles.
We also need to transpose the metadata file in order to merge it with the ASV table.

In [81]:
#Select only Cycles in the metadata file
md <- subset(metadata, ATTRIBUTE_Sample_Type == "Cycle")

In [82]:
#Transpose metadata file 
rownames(md) <- md$filename
md <- select(md, -c(filename))
md <- (t(md))

In [83]:
#Fix rownames to column, create dataframe
Feature.ID <- rownames(md)
rownames(md) <- NULL
md <- cbind(Feature.ID, md)
md <- as.data.frame(md)

In [84]:
# Make table18SV9 characters to match metadata file to allow them to merge
table18SV9 <- table18SV9 %>% mutate_if(is.numeric, as.character)

In [85]:
#Remove files that don't have both metadata and ASV files associated with itself
common_column_names <- intersect(names(table18SV9), names(md))
table18SV9 <- table18SV9[, common_column_names]
md <- md[, common_column_names]

In [86]:
head(table18SV9, 3)
head(md,3)

,Feature.ID,CCE_P1706_39.mzxml,CCE_P1706_41.mzxml,CCE_P1706_43.mzxml,CCE_P1706_45.mzxml,CCE_P1706_47.mzxml,CCE_P1706_49.mzxml,CCE_P1706_51.mzxml,CCE_P1706_53.mzxml,CCE_P1706_55.mzxml,⋯,CCE_P1706_210.mzxml,CCE_P1706_212.mzxml,CCE_P1706_214.mzxml,CCE_P1706_216.mzxml,CCE_P1706_218.mzxml,CCE_P1706_220.mzxml,CCE_P1706_222.mzxml,CCE_P1706_224.mzxml,CCE_P1706_228.mzxml,CCE_P1706_230.mzxml
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,16e3d4f1cb72c753aa6420206110abec,114,161,399,2776,5732,3550,80,128,942,⋯,748,1224,2744,1023,132,286,644,1835,6,13
2,6fdbb8c63fe109f1beeb14f76c30e678,19,69,214,920,2517,1223,126,75,240,⋯,400,2393,1101,625,25,271,438,785,20,35
3,d1b754a61547771e95bc457845b76031,114,69,381,1169,15012,13715,142,124,365,⋯,45,277,201,425,80,10,89,90,0,23


,Feature.ID,CCE_P1706_39.mzxml,CCE_P1706_41.mzxml,CCE_P1706_43.mzxml,CCE_P1706_45.mzxml,CCE_P1706_47.mzxml,CCE_P1706_49.mzxml,CCE_P1706_51.mzxml,CCE_P1706_53.mzxml,CCE_P1706_55.mzxml,⋯,CCE_P1706_210.mzxml,CCE_P1706_212.mzxml,CCE_P1706_214.mzxml,CCE_P1706_216.mzxml,CCE_P1706_218.mzxml,CCE_P1706_220.mzxml,CCE_P1706_222.mzxml,CCE_P1706_224.mzxml,CCE_P1706_228.mzxml,CCE_P1706_230.mzxml
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,ATTRIBUTE_x,-101.419,-101.419,-101.419,-101.419,-101.419,-101.419,-103.515,-103.515,-103.515,⋯,-141.430,-141.430,-141.430,-141.430,-141.430,-139.636,-139.636,-139.636,-139.636,-139.636
2,ATTRIBUTE_y,101.9560,101.9560,101.9560,101.9560,101.9560,101.9560,100.0030,100.0030,100.0030,⋯,86.9585,86.9585,86.9585,86.9585,86.9585,88.6513,88.6513,88.6513,88.6513,88.6513
3,ATTRIBUTE_z,109.445,110.445,111.444,111.884,112.344,112.786,109.445,110.445,111.444,⋯,110.445,111.444,111.884,112.344,112.786,109.445,110.445,111.444,112.344,112.786


Now we can merge the taxonomic information and ASV table by Feature.ID
We also can merge the metadata file now.

In [87]:
#Merge two datafiles by Feature.ID
ASV18SV9 <- merge(table18SV9, ASVtax18SV9, by = 'Feature.ID')
#Merge metadata file and ASV file
ASV18SV9 <- bind_rows(ASV18SV9, md)

head(ASV18SV9,3)

,Feature.ID,CCE_P1706_39.mzxml,CCE_P1706_41.mzxml,CCE_P1706_43.mzxml,CCE_P1706_45.mzxml,CCE_P1706_47.mzxml,CCE_P1706_49.mzxml,CCE_P1706_51.mzxml,CCE_P1706_53.mzxml,CCE_P1706_55.mzxml,⋯,CCE_P1706_214.mzxml,CCE_P1706_216.mzxml,CCE_P1706_218.mzxml,CCE_P1706_220.mzxml,CCE_P1706_222.mzxml,CCE_P1706_224.mzxml,CCE_P1706_228.mzxml,CCE_P1706_230.mzxml,Taxon,Confidence
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
1,000167ce642266c21c62843e4a010266,0,0,0,0,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,Eukaryota;Rhizaria;Radiolaria;Acantharea;Acantharea_X;Acantharea_XX;Acantharea_XXX;Acantharea_XXX_sp.;,0.9991760
2,001567b7cb76eecd73f31a288fcfcb69,0,0,0,2,0,0,0,0,0,⋯,0,0,0,0,0,0,0,0,Eukaryota;Stramenopiles,0.7359983
3,0018f741aa09de859e5af52f19a2b7c1,0,0,0,0,0,0,0,0,0,⋯,0,3,0,0,0,0,0,0,Eukaryota,1.0000000


Create 'shared name' header for Cytoscape and deconcatenate the Taxon Column:

In [88]:
#create a shared name column for later Cytoscape processing
ASV18SV9$'shared name' <- ASV18SV9$Taxon

In [89]:
#Deconcatenate 
ASV18SV9 <- separate(data = ASV18SV9, col = Taxon, into = c('Domain18SV9', 'Kingdom18SV9', 
                                                            'Phylum18SV9', 'Class18SV9', 
                                                            'Order18SV9', 'Family18SV9',
                                                            'Genus18SV9', 'Species18SV9'), sep = ";")

Warning message:
"Expected 8 pieces. Additional pieces discarded in 3835 rows [1, 6, 7, 10, 11, 12, 16, 17, 19, 20, 22, 24, 31, 32, 36, 37, 41, 43, 45, 50, ...]."
Warning message:
"Expected 8 pieces. Missing pieces filled with `NA` in 6447 rows [2, 3, 4, 5, 8, 9, 13, 14, 15, 18, 21, 23, 25, 26, 27, 28, 29, 30, 33, 34, ...]."


In [90]:
#Write into .csv file
write.csv(ASV18SV9, 'ASV18SV9_With_Metadata_dups.csv')

## Step 4: no duplicate files for 16S and 18SV9

ASV16S First:

In [91]:
#Create matrix to remove duplicate samples created in Step2, remove BBL and SBB samples from name index
removedups <- nameindex %>% group_by(name2) %>% slice(-1)
removedups <- removedups %>% filter(!grepl('_B', name1))
removedups <- removedups %>% filter(!grepl('_SBB', name1))

In [92]:
#Remove duplicate samples, and patch together dataframe to contain Feature and taxonomy info (make sure column numbers are correct)
names.use <- names(ASV16S)[(names(ASV16S) %in% removedups$name4)]
ASV16S_nodups <- ASV16S[, c('Feature.ID', names.use, colnames(ASV16S[,144:152]))]

In [93]:
#Write csv
write.csv(ASV16S_nodups, 'ASV16S_With_Metadata_no_dups.csv')

ASV18SV9 Next:

In [94]:
#Needs the removedups and names.use files from the previous step
ASV18SV9_nodups <- ASV18SV9[, c('Feature.ID', names.use, colnames(ASV18SV9[,144:153]))]

In [95]:
write.csv(ASV18SV9_nodups, 'ASV18SV9_With_Metadata_no_dups.csv')

In [98]:
sessionInfo()

R version 4.2.2 (2022-10-31 ucrt)
Platform: x86_64-w64-mingw32/x64 (64-bit)
Running under: Windows 10 x64 (build 22000)

Matrix products: default

locale:
[1] LC_COLLATE=English_United Kingdom.utf8 
[2] LC_CTYPE=English_United Kingdom.utf8   
[3] LC_MONETARY=English_United Kingdom.utf8
[4] LC_NUMERIC=C                           
[5] LC_TIME=English_United Kingdom.utf8    

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
[1] tidyr_1.2.1  dplyr_1.0.10

loaded via a namespace (and not attached):
 [1] magrittr_2.0.3   tidyselect_1.2.0 uuid_1.1-0       R6_2.5.1        
 [5] rlang_1.0.6      fastmap_1.1.0    fansi_1.0.3      tools_4.2.2     
 [9] utf8_1.2.2       DBI_1.1.3        cli_3.4.1        withr_2.5.0     
[13] ellipsis_0.3.2   htmltools_0.5.3  assertthat_0.2.1 digest_0.6.30   
[17] tibble_3.1.8     lifecycle_1.0.3  crayon_1.5.2     IRdisplay_1.1   
[21] purrr_0.3.5      repr_1.1.5       base64enc_0.1-3  vctrs